In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## check if page downloaded successfully

In [2]:
url = "https://www.dli.pa.gov/Individuals/Workforce-Development/warn/notices/Pages/default.aspx"
page = requests.get(url)
page.status_code

200

## webpage url parse

In [3]:
soup = BeautifulSoup(page.text, 'html.parser')
data = soup.findAll('div',attrs={'class':'ms-rtestate-field'})
month_links = []
for div in data:
    links = div.findAll('a')
    for a in links:
        month_links.append("https://www.dli.pa.gov/"+a['href'])

In [4]:
len(month_links)

110

In [5]:
#month_links

## Data Collection

In [6]:
def get_df(url):
    #
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    table_data = soup.findAll('td')
    #
    company_name = []
    company_address = []
    country_l = []
    employees_affected_l = []
    effective_date_l = []
    closure_or_layoff_l = []
    
    #
    error_link = []
    error_name = []

    for ele in table_data:#[10:11]:
        data = ele.find_all('p')
        if data:
            try:
                name = data[0].find('strong').text
            except: 
                continue
            if len(data) != 2:
                error_name.append(name)
                error_link.append(url)
                continue
            if name in company_name:
                continue
            ad_list = []
            #delete the name part
            ad_list.append(", ".join([ele.strip() for ele in re.findall(r'\>([^\>]*)\<', str(data[0]).split("</strong>")[1]) if ele not in ('\n','')]))

            for ad in data[1:-1]:

                ad_list.append(", ".join([ele.strip() for ele in re.findall(r'\>([^\>]*)\<', str(ad)) if ele not in ('\n','')]))

            part1 = " and ".join(ad_list)

            #print(part1)

            part2 = " ".join([ele.strip() for ele in re.findall(r'\>([^\>]*)\<', str(data[-1])) if ele not in ('\n','')])
            try:
                string = part1+" "+part2


                address = string.split("COUNTY:")[0]

                country = string.split("COUNTY:")[1].split("# AFFECTED:")[0]

                string = string.replace("LAYOFF EFFECTIVE DATES:", "EFFECTIVE DATE:")

                employees_affected = string.split("COUNTY:")[1].split("# AFFECTED:")[1].split("EFFECTIVE DATE:")[0]



                effective_date = string.split("COUNTY:")[1].split("# AFFECTED:")[1].split("EFFECTIVE DATE:")[1].split("CLOSURE OR LAYOFF:")[0]

                string = string.replace("CLOSING OR LAYOFF:", "CLOSURE OR LAYOFF:")

                closure_or_layoff = string.split("COUNTY:")[1].split("# AFFECTED:")[1].split("EFFECTIVE DATE:")[1].split("CLOSURE OR LAYOFF:")[1]

                company_name.append(name)

                company_address.append(address)

                country_l.append(country)

                employees_affected_l.append(employees_affected)

                effective_date_l.append(effective_date)

                closure_or_layoff_l.append(closure_or_layoff)
                
            except:
                
                error_name.append(name)
                error_link.append(url)
                
    #create dataframe
    d1 = {'company_name': company_name, 'company_address': company_address, 'country': country_l, 'employees_affected': employees_affected_l, 'effective_date': effective_date_l, 'closure_or_layoff': closure_or_layoff_l}
    df1 = pd.DataFrame(data=d1)
    
    #create error dataframe
    d2 = {'links': error_link, 'name': error_name}
    df2 = pd.DataFrame(data=d2)
    df2 = df2.drop_duplicates()
    print(url.split('/')[-1].split('.')[0]+' is done!')
    return df1, df2
df_lists = []
df_error_lists = []
for ele in month_links[:14]:
    try:
        result = get_df(ele)
        df_lists.append(result[0])
        df_error_lists.append(result[1])
    except:
        print(ele)
df = pd.concat(df_lists)
df.to_csv("final_result.csv", index = False)
df = pd.concat(df_error_lists)
df.to_csv("final_error.csv", index = False)

February-2020 is done!
January-2020 is done!
December-2019 is done!
November-2019 is done!
October-2019 is done!
September-2019 is done!
August-2019 is done!
July-2019 is done!
June-2019 is done!
May-2019 is done!
April-2019 is done!
March-2019 is done!
February-2019 is done!
January-2019 is done!
